# INFO 370 Problem Set 3: Webscraping
*Name:* Israel Martinez

## 1 Legal and Ethical Issues
Before you can start scraping recipes, you have to find a webpage where webscraping is legal. In recentyears it is getting more and more common that the sites explicitly ban it. For instance,allrecipies.comstates in Terms of Use that:

(e) you shall not use any manual or automated software, devices or other processes(including but not limited to spiders, robots, scrapers, crawlers, avatars, data mining toolsor the like) to “scrape” or download data from the Services. . .

Some websites permit downloading for “personal non-commercial use”. Some websites stay silent aboutscraping. I think what you will do will be “personal non-commercial use” but I am not a lawyer...Here are some recipe websites that do not ban scraping:

• What’s in the pan?https://whatsinthepan.com/seems not to mention scraping at all

• Feasting at homehttps://www.feastingathome.com/tells not to re-publish recepes, not evenon social media, but does not mention scraping.

• You may find recipes on github too. Github “Acceptable Use Policies” state thatYou may scrape the website for the following reasons:–Researchers may scrape public, non-personal information from the Service for re-search purposes, only if any publications resulting from that research are openaccess.–Archivists may scrape the Service for public data for archival purposes.My reading is the education is OK too but I am not a lawyer...

• Wiki Cookbook:https://en.wikibooks.org/wiki/Cookbook:Recipes

1. (8pt) Find a website that you consider scraping.

I will be using the [Wiki Cookbook](https://en.wikibooks.org/wiki/Cookbook:Recipes).

2. (11pt) Discuss the legality of scraping this site: does it mention data collection in its terms-ofservice? Does robots.txt file allow/disallow what you want to do?

After looking at Wikipedia's website for any mention of data collection in its terms-of-service, I see that there is a Create Commons License which enables the free distribution of otherwise copyrighted "work." After glancing at the sites robots.txt, it looks like it would not prevent me from doing what I want to do in this assignment.

3. (11pt) Discuss what kind of steps do you take to lessen the burden to the provider.

To lessen the burden, I plan to donate money to Wikipedia as a "thank you" for the open access. I will do this as soon as Wikipedia does their once a year alert to all site visitors to donate some amount like $3.

## 2 Scrape the recipies (60pt)

Your coding task is to scrape the website and collect all the ingredients for at least 50 recipes.

1. Write the code to scrape (at least 50) recipes from the webpage you chose.

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
url = "https://en.wikibooks.org/wiki/Cookbook:Recipes"
response = requests.get(url)
# Response is 200, meaning successful
response.status_code

200

In [3]:
wiki = BeautifulSoup(response.content, "html.parser").body
wiki.name

'body'

In [45]:
# finds all lists linked to a recipe
links = wiki.find_all("ul")[3]

2. Pull out the list of ingredients, and split those into quantities, units, and ingredients as well as you can.

Hint: sometimes you find unstructured ingredient lines. In that case there is little you can do with it. It is extremely hard to tell what is what in “a handful of sugar peas” or how much exactly is “a bunch” of finely chopped cilantro.

In [46]:
from urllib.request import urljoin

In [48]:
# These are the steps I did to get all recipes
# 1. Get all list items.
# 2. Get the full recipe link.
# 3. Create list which will later store the ingredients.
# 4. Find all headers which represent the ingredients name and assign
#    to list
# 5. Get all items in the list
# 6. Loop through each item list
# 7. Create new recipe list which assigns the text, amount, unit, and 
#    ingredient in the form.
# 8. Append text to new recipe.
# 9. Split items into array to be iterated through.
# 10. Apend each to the new recipe list.
# 11. Add to the final ingredient list of the recipe
# 12. Assign list of ingredients to the recipes dictionary

# dict to store the names and ingredients of my recipes
recipes = {}
baseURL = 'https://en.wikibooks.org/'

for i in range(0, 51):
    # (1)
    tag = links.find_all('li')[i]
    linkName = tag.a['href']
    # (2)
    urlRecipe = urljoin(baseURL, linkName)
    response = requests.get(urlRecipe)
    
    # (3)
    ingList = []
    r2 = BeautifulSoup(response.content, "html.parser")
    # (4)
    name = r2.find('h1').text
    # (5)
    recipeList = r2.find_all('ul')
    
    # (6)
    rec2 = recipeList[0].find_all('li')
    
    for item in rec2:
        # (7)
        recipeNew = []
        item2 = item.text
        # (8)
        recipeNew.append(item2)
        # (9)
        item3 = item2.split(" ", 2)
        # (10)
        for word in item3:
            recipeNew.append(word)
        # (11)
        ingList.append(recipeNew)
    # (12)
    recipes[name] = ingList
recipes

{'Cookbook:Hard Tack of India (Baati)': [['1 pound (450 g) wheat flour',
   '1',
   'pound',
   '(450 g) wheat flour'],
  ['1/2 teaspoon salt', '1/2', 'teaspoon', 'salt'],
  ['3 tablespoons oil', '3', 'tablespoons', 'oil'],
  ['1 1/2 teaspoons ghee (for serving)',
   '1',
   '1/2',
   'teaspoons ghee (for serving)']],
 'Cookbook:Baba Ganoush': [['1 medium-large eggplant, any variety, 1 to 1½ pounds',
   '1',
   'medium-large',
   'eggplant, any variety, 1 to 1½ pounds'],
  ['2 tablespoons raw tahini', '2', 'tablespoons', 'raw tahini'],
  ['Juice of 1 lime', 'Juice', 'of', '1 lime'],
  ['1 clove garlic, crushed', '1', 'clove', 'garlic, crushed'],
  ['3 tablespoons olive oil', '3', 'tablespoons', 'olive oil'],
  ['½ tsp salt', '½', 'tsp', 'salt'],
  ['Paprika or cayenne pepper, as a garnish',
   'Paprika',
   'or',
   'cayenne pepper, as a garnish']],
 'Cookbook:Salt Cod and Potato Casserole (Bacalhau à Gomes de Sá)': [['2 lbs. (900 g) salt cod',
   '2',
   'lbs.',
   '(900 g) salt cod']

3. Store the ingredients from each recipe in a data frame. The data frame should contain the following variables:

id - recipe id (just a number)

name - recipe name

text - the ingredient line as text as printed on the webpage, stripped of all html attributes

quantity - quantity of the ingredient

unit - unit of the quantity

ingredient - the name/explanation of the ingredient

The data frame should be in long form, i.e. one line for each ingredient, and several lines for each recipe.

In [56]:
import numpy as np
import pandas as pd

# These are the steps I took to store my ingredients from each recipe
# in a data frame:
# 1. Create empty lists which will represent each column.
# 2. Loop through each key in recipes.
# 3. Assign each recipe a number for identification purposes.
# 4. For each item in key, add them to their respective column lists.
# 5. For assigning list columns: key is name, item 0 is the text,
#    item 1 is quanity, item 2 is unit, and item 3 is ingredient.
# 6. Create new recipes dictionary which assigns the lists to column
#    keys.

# (1)
IDs = []
names = []
texts = []
quantities = []
units = []
ingredients = []
idNum = 0

# (2)
for key in recipes:
    # (3)
    idNum = idNum + 1
    # (4)
    for row in recipes[key]:
        # (5)
        if(len(row) == 4):
            names.append(key)
            texts.append(row[0])
            quantities.append(row[1])
            units.append(row[2])
            ingredients.append(row[3])
            IDs.append(idNum)

# (6)    
recipeDict = {'ID': IDs, 'Name': names, 'Text': texts, 
              'Quantity': quantitys, 'Unit': units, 
              'ingredient': ingredient}

4. Store your dataframe in a file so you don’t have to repeat scraping.

In [72]:
dfRecipes = pd.DataFrame(recipeDict)

# This code is not creating a file at this location and I'm not sure why.
dfRecipes.to_csv(r'D:\Users\israe\Documents\ACADEMICS\4th year\INFO 370\ps03\recipes.csv', index=False)
dfRecipes

,ID,Name,Text,Quantity,Unit,ingredient
0,1,Cookbook:Hard Tack of India (Baati),1 pound (450 g) wheat flour,1,pound,(450 g) wheat flour
1,1,Cookbook:Hard Tack of India (Baati),1/2 teaspoon salt,1/2,teaspoon,salt
2,1,Cookbook:Hard Tack of India (Baati),3 tablespoons oil,3,tablespoons,oil
3,1,Cookbook:Hard Tack of India (Baati),1 1/2 teaspoons ghee (for serving),1,1/2,teaspoons ghee (for serving)
4,2,Cookbook:Baba Ganoush,"1 medium-large eggplant, any variety, 1 to 1½ ...",1,medium-large,"eggplant, any variety, 1 to 1½ pounds"
...,...,...,...,...,...,...
313,50,Cookbook:Bruschetta,Extra-virgin olive oil,Extra-virgin,olive,oil
314,50,Cookbook:Bruschetta,Toppings of your choice (see examples below),Toppings,of,your choice (see examples below)
315,51,Cookbook:Bubble and Squeak,"1 medium onion, chopped, optional, red, white,...",1,medium,"onion, chopped, optional, red, white, or yello..."
316,51,Cookbook:Bubble and Squeak,1 tablespoon fat or oil,1,tablespoon,fat or oil


## 3 What is more common: sweet or salty food? (10pt)

Now let’s do some analysis based on the data you got. Are there more salty foods or sweet foods?

1. For each recipe, find if it contains salt, and if it contains sugar. Are there more recipes with salt or more with sugar?

Hint: check out functions: pd.Series.str.contains and np.any.

In [61]:
# Steps to finding how much salt and sugar is in the recipes:
# 1. Assign number of sugars and salts to empty variables
# 2. Loop through the ingredient column in the recipes data frame
# 3. Add 1 to the variables for each occurrence.
# 4. Print results.

# (1)
sugars = 0
salts = 0

# (2)
for ingred in dfRecipes.ingredient:
    # (3)
    if "sugar" in ingred:
        sugars = sugars + 1
    elif "salt" in ingred:
        salts = salts + 1

# (4)
print("sugar counts:", sugars)
print("salt counts:", salts)

sugar counts: 13
salt counts: 16


It appears that there are more counts of salt than there are sweets in recipes.

2. But what about the absolute quantities? Do all these recipes contain more salt or more sugar? Add the salt and sugar quantities over all recipes and see which one wins.

If you cannot do this, explain what are the problems!

In [63]:
sugarAmt = 0
saltAmt = 0

for ingred in dfRecipes.ingredient:
    if "sugar" in ingredient:
        sugarAmt += dfRecipes.quantity
    elif "salt" in ingredient:
        saltAmt += dfRecipes.quantity
        
print("sugar quantities:", sugarAmt)
print("salt quanitities:", saltAmt)

sugar quantities: 0
salt quanitities: 0


I tried to solve this problem by using a similar format as I did for # 1, but I ran into a few problems. The first problem I saw was that not all quantities in the quantity column is an integer (some values are strings like "Extra-virgin" and "Toppings"), so these values couldn't be added. However, I'm not sure why I didn't run into an error or why it didn't attempt to add anything to my variables, since I am referencing the column to add. Also, each unit has a different measurement such as "pound" and "teaspoon", so I wasn't sure how to convert that to a standard measure for an accurate count.

## 4 Finally...

I spent 10 hours on this assignment.